In [11]:
import torch
from tqdm import tqdm
from PIL import Image
from torch import cat as torch_cat
import pandas as pd
import torch.nn.functional as F

In [12]:
pip install optuna

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import AutoModel, AutoTokenizer, ViTModel, ViTConfig
import optuna
import pandas as pd
from PIL import Image
import os
import numpy as np
import random
import logging
import pickle

In [14]:
#path to Pickle file 
input_path = '/imaging/students/Abhi/project/df_with_captions.pkl'

#loading the dataframe from the pickle file
with open(input_path, 'rb') as file: 
    df_with_captions = pickle.load(file)
    
print(df_with_captions.shape)

(423256, 3)


In [15]:
print(df_with_captions['Path'][0])

CheXpert-v1.0-small/train/patient00001/study1/view1_frontal.jpg


In [16]:
df_with_captions['Path'] = df_with_captions['Path'].str.replace('CheXpert-v1.0-small', '/imaging/students/Abhi/project/CheXpert_dataset')

In [17]:
sample_df2 = df_with_captions[100000:160001]

In [18]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [20]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import AutoModel, AutoTokenizer, ViTModel, ViTConfig
import optuna
import pandas as pd
from PIL import Image
import os
import numpy as np
import random
import logging
from tqdm.auto import tqdm  # Import TQDM for progress bar
from torch.cuda.amp import autocast, GradScaler  # For mixed precision

# Set up logging
log_dir = "log_hyper_towardsFSM"
log_file = os.path.join(log_dir, "hyperparameter_tuning_log.txt")

# Ensure the directory exists
if not os.path.exists(log_dir):
    os.makedirs(log_dir)  # Create the directory if it doesn't exist

# Remove the existing log file if it exists
if os.path.exists(log_file):
    os.remove(log_file)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define image transformations (t_v)
def enhance_contrast(image):
    np_image = np.array(image)
    min_val, max_val = np.percentile(np_image, (2, 98))
    np_image = np.clip(np_image, min_val, max_val)
    np_image = (np_image - min_val) / (max_val - min_val)
    return Image.fromarray((np_image * 255).astype(np.uint8))

# Function to randomly sample and apply a subset of transformations
def random_transform():
    # Always apply Resize to ensure all images have the same size
    final_transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Ensures all images are the same size
        transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
        transforms.RandomRotation(degrees=15),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    return final_transform

# Define text transformations (t_u)
def text_transforms(text, max_length=128):
    return tokenizer(text, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')

# Define custom dataset using sample_df
class CheXpertDataset(Dataset):
    def __init__(self, df):
        self.df = df  # sample_df is passed here

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Get the image path and the caption from the dataframe
        img_path = self.df.iloc[idx]['Path']  # Directly using the full image path from sample_df['Path']
        image = Image.open(img_path).convert('RGB')

        # Apply a random subset of transformations
        transform = random_transform()
        image = transform(image)

        # Get the caption from the dataframe
        text = self.df.iloc[idx]['Captions']

        return {'image': image, 'text': text}

# Define the image encoder (ViT)
class ImageEncoder(nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        config = ViTConfig.from_pretrained('google/vit-base-patch16-224')
        config.output_hidden_states = False
        self.vit = ViTModel(config)
        self.fc = nn.Linear(config.hidden_size, output_dim)

    def forward(self, x):
        outputs = self.vit(x)
        return self.fc(outputs.last_hidden_state[:, 0, :])

# Define the text encoder (CXR-BERT-specialized)
class TextEncoder(nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        self.bert = AutoModel.from_pretrained("microsoft/BiomedVLP-CXR-BERT-specialized", trust_remote_code=True)
        self.fc = nn.Linear(self.bert.config.hidden_size, output_dim)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return self.fc(outputs.last_hidden_state[:, 0, :])

# Define the ConVIRT model
class ConVIRT(nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        self.image_encoder = ImageEncoder(output_dim)
        self.text_encoder = TextEncoder(output_dim)

    def forward(self, images, input_ids, attention_mask):
        image_embeddings = self.image_encoder(images)
        text_embeddings = self.text_encoder(input_ids, attention_mask)
        return image_embeddings, text_embeddings

# Define the contrastive loss
class ContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.07):  # Fixed temperature
        super().__init__()
        self.temperature = temperature
        self.cosine_similarity = nn.CosineSimilarity(dim=-1)
        self.cross_entropy = nn.CrossEntropyLoss()

    def forward(self, image_features, text_features):
        batch_size = image_features.shape[0]
        similarities = self.cosine_similarity(image_features.unsqueeze(1), text_features.unsqueeze(0)) / self.temperature
        labels = torch.arange(batch_size).long().to(similarities.device)
        loss_i2t = self.cross_entropy(similarities, labels)
        loss_t2i = self.cross_entropy(similarities.t(), labels)
        return (loss_i2t + loss_t2i) / 2

# Define the training function with mixed precision and TQDM progress bar for epochs
def train(model, train_loader, optimizer, loss_fn, device, num_epochs):
    model.train()
    scaler = GradScaler()  # Initialize gradient scaler for mixed precision
    total_loss = 0
    progress_bar = tqdm(range(num_epochs), desc="Training Epochs")  # Progress bar for epochs

    for epoch in progress_bar:
        epoch_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):  # Progress bar for batches
            images, texts = batch['image'].to(device), batch['text']
            text_inputs = text_transforms(texts)
            text_inputs = {k: v.squeeze(1).to(device) for k, v in text_inputs.items()}

            optimizer.zero_grad()

            # Mixed precision training
            with autocast():
                image_embeddings, text_embeddings = model(images, text_inputs['input_ids'], text_inputs['attention_mask'])
                loss = loss_fn(image_embeddings, text_embeddings)

            # Backward pass with mixed precision scaling
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(train_loader)
        progress_bar.set_postfix(loss=avg_loss)  # Show average loss for each epoch

        # Clear GPU cache after each epoch
        torch.cuda.empty_cache()

        # Log memory usage after each epoch
        logger.info(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss}")
        logger.info(f"Memory Usage: {torch.cuda.memory_allocated(device) / (1024 ** 3):.2f} GB")

    return avg_loss

# Define the objective function for Optuna with TQDM progress bar
def objective(trial):
    # Hyperparameters to optimize
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    batch_size = trial.suggest_int('batch_size', 16, 32)  # Allows any integer between 16 and 32
    temperature = trial.suggest_uniform('temperature', 0.01, 0.1)  # Tune temperature between 0.01 and 0.1

    # Fixed number of epochs
    num_epochs = 20  # Set epochs to 20
    
    # Model and optimizer
    model = ConVIRT(output_dim=512).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = ContrastiveLoss(temperature=temperature).to(device)  # Pass the tuned temperature

    # Data loader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training loop with the fixed number of epochs (20)
    avg_loss = train(model, train_loader, optimizer, loss_fn, device, num_epochs)

    # Log each trial's hyperparameters and results
    with open(log_file, "a") as f:
        f.write(f"Trial {trial.number} | lr: {lr:.6f}, batch_size: {batch_size}, temperature: {temperature:.4f}, loss: {avg_loss:.4f}\n")

    return avg_loss

# Main execution
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Use a smaller subset of 20% of the data for tuning
    train_subset = sample_df2.sample(frac=0.3)  # Use 10% of your data
    train_dataset = CheXpertDataset(df=train_subset)

    # Initialize DataLoader for batching
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Create the tokenizer for CXR-BERT-specialized
    tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedVLP-CXR-BERT-specialized", trust_remote_code=True)

    # Run Optuna optimization to find the best hyperparameters with TQDM progress bars
    total_trials = 40  # Reduced total number of trials
    with tqdm(total=total_trials, desc="Hyperparameter Tuning", unit="trial") as pbar:  # Set total trials with ETA
        def callback(study, trial):
            pbar.update(1)  # Update progress bar after each trial
            pbar.set_postfix({"Best Value": study.best_value})  # Show best value in progress bar

        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=total_trials, callbacks=[callback])  # Reduced number of trials to 40

    # Print the best parameters and results
    logger.info("Best trial:")
    trial = study.best_trial
    logger.info(f"  Value: {trial.value:.4f}")
    logger.info("  Params: ")
    for key, value in trial.params.items():
        logger.info(f"    {key}: {value}")

    # Extract the best hyperparameters from the study
    best_params = trial.params
    best_lr = best_params['lr']
    best_batch_size = best_params['batch_size']
    best_temperature = best_params['temperature']

    # Perform final training using the best hyperparameters
    logger.info("Starting final training with best hyperparameters...")

    # Re-create DataLoader with the best batch size
    train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True)

    # Initialize the model with the fixed output dimension
    model = ConVIRT(output_dim=512).to(device)

    # Initialize optimizer with the best learning rate
    optimizer = optim.Adam(model.parameters(), lr=best_lr)

    # Initialize the loss function with the best temperature
    loss_fn = ContrastiveLoss(temperature=best_temperature).to(device)

    # Final training loop with the fixed number of epochs (30)
    train(model, train_loader, optimizer, loss_fn, device, 30)

    logger.info("Final training completed.")


Hyperparameter Tuning:   0%|          | 0/40 [00:00<?, ?trial/s]

[I 2024-10-13 14:21:10,531] A new study created in memory with name: no-name-29cd80ad-65d0-411b-a938-f3ccfb22fc01
/tmp/ipykernel_33494/1694465095.py:177: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
/tmp/ipykernel_33494/1694465095.py:179: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  temperature = trial.suggest_uniform('temperature', 0.01, 0.1)  # Tune temperature between 0.01 and 0.1
/tmp/ipykernel_33494/1694465095.py:137: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # Initialize gradient scaler for mixed precision


Training Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1/20:   0%|          | 0/581 [00:00<?, ?it/s]

/tmp/ipykernel_33494/1694465095.py:151: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
INFO:__main__:Epoch 1/20, Loss: 3.433565288320466
INFO:__main__:Memory Usage: 2.94 GB


...

INFO:__main__:Epoch 28/30, Loss: 0.8327246904902988
INFO:__main__:Memory Usage: 2.94 GB


Epoch 29/30:   0%|          | 0/1125 [00:00<?, ?it/s]

INFO:__main__:Epoch 29/30, Loss: 0.7968020991484324
INFO:__main__:Memory Usage: 2.94 GB


Epoch 30/30:   0%|          | 0/1125 [00:00<?, ?it/s]

INFO:__main__:Epoch 30/30, Loss: 0.7680052935679753
INFO:__main__:Memory Usage: 2.94 GB
INFO:__main__:Final training completed.


In [10]:
!nvidia-smi

Sun Oct 13 14:14:57 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA TITAN V                 Off |   00000000:03:00.0  On |                  N/A |
| 28%   42C    P8             29W /  250W |     599MiB /  12288MiB |     38%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [23]:
# Specify the path to the file
file_path = 'log_hyper_towardsFSM/hyperparameter_tuning_log.txt'

# Open and read the file
try:
    with open(file_path, 'r') as file:
        file_content = file.read()

    # Display the content of the file
    print(file_content)
except FileNotFoundError:
    print(f"File not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")


Trial 0 | lr: 0.002124, batch_size: 31, temperature: 0.0774, loss: 3.4332
Trial 1 | lr: 0.000169, batch_size: 22, temperature: 0.0830, loss: 3.0890
Trial 2 | lr: 0.006870, batch_size: 16, temperature: 0.0700, loss: nan
Trial 3 | lr: 0.000106, batch_size: 24, temperature: 0.0561, loss: 3.1781
Trial 4 | lr: 0.000086, batch_size: 29, temperature: 0.0131, loss: 3.3668
Trial 5 | lr: 0.000118, batch_size: 18, temperature: 0.0820, loss: 2.8906
Trial 6 | lr: 0.001994, batch_size: 32, temperature: 0.0840, loss: 3.4645
Trial 7 | lr: 0.001810, batch_size: 24, temperature: 0.0121, loss: 3.1781
Trial 8 | lr: 0.000017, batch_size: 16, temperature: 0.0104, loss: 1.1228
Trial 9 | lr: 0.000109, batch_size: 25, temperature: 0.0378, loss: 3.2189
Trial 10 | lr: 0.000011, batch_size: 21, temperature: 0.0506, loss: 1.2426
Trial 11 | lr: 0.000013, batch_size: 16, temperature: 0.0279, loss: 1.0479
Trial 12 | lr: 0.000010, batch_size: 16, temperature: 0.0297, loss: 1.0492
Trial 13 | lr: 0.000027, batch_size: 1

In [22]:
pwd

'/imaging/students/Abhi/project'